# Механизмы внимания и трансформеры

Одним из главных недостатков рекуррентных сетей является то, что все слова в последовательности оказывают одинаковое влияние на результат. Это приводит к снижению эффективности стандартных моделей LSTM-энкодер-декодер для задач преобразования последовательностей, таких как распознавание именованных сущностей и машинный перевод. На практике отдельные слова во входной последовательности часто оказывают большее влияние на выходные данные, чем другие.

Рассмотрим модель преобразования последовательностей, например, машинный перевод. Она реализуется с помощью двух рекуррентных сетей, где одна сеть (**энкодер**) сжимает входную последовательность в скрытое состояние, а другая сеть (**декодер**) разворачивает это скрытое состояние в переведенный результат. Проблема такого подхода заключается в том, что финальному состоянию сети сложно запомнить начало предложения, что приводит к снижению качества модели на длинных предложениях.

**Механизмы внимания** предоставляют способ взвешивания контекстного влияния каждого входного вектора на каждое предсказание RNN. Это реализуется путем создания "ярлыков" между промежуточными состояниями входной RNN и выходной RNN. Таким образом, при генерации выходного символа $y_t$ мы будем учитывать все скрытые состояния входа $h_i$ с различными весовыми коэффициентами $\alpha_{t,i}$.

![Изображение, показывающее модель энкодер/декодер с аддитивным слоем внимания](../../../../../translated_images/encoder-decoder-attention.7a726296894fb567aa2898c94b17b3289087f6705c11907df8301df9e5eeb3de.ru.png)  
*Модель энкодер-декодер с механизмом аддитивного внимания из [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитируется из [этого блога](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матрица внимания $\{\alpha_{i,j}\}$ представляет степень, с которой определенные входные слова влияют на генерацию конкретного слова в выходной последовательности. Ниже приведен пример такой матрицы:

![Изображение, показывающее пример выравнивания, найденного RNNsearch-50, взято из Bahdanau - arviz.org](../../../../../translated_images/bahdanau-fig3.09ba2d37f202a6af11de6c82d2d197830ba5f4528d9ea430eb65fd3a75065973.ru.png)  

*Рисунок взят из [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Рис.3)*

Механизмы внимания ответственны за многие современные или близкие к современным достижениям в области обработки естественного языка. Однако добавление внимания значительно увеличивает количество параметров модели, что привело к проблемам масштабирования с RNN. Одним из ключевых ограничений масштабирования RNN является то, что рекуррентная природа моделей затрудняет пакетную обработку и параллелизацию обучения. В RNN каждый элемент последовательности должен обрабатываться в последовательном порядке, что делает параллелизацию сложной.

Применение механизмов внимания в сочетании с этим ограничением привело к созданию современных трансформерных моделей, которые мы знаем и используем сегодня, таких как BERT и OpenGPT3.

## Трансформерные модели

Вместо передачи контекста каждого предыдущего предсказания на следующий шаг оценки, **трансформерные модели** используют **позиционные кодировки** и внимание для захвата контекста заданного входа в пределах предоставленного окна текста. На изображении ниже показано, как позиционные кодировки с вниманием могут захватывать контекст в пределах заданного окна.

![Анимация, показывающая, как выполняются оценки в трансформерных моделях.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Поскольку каждая входная позиция отображается независимо на каждую выходную позицию, трансформеры могут лучше параллелизоваться, чем RNN, что позволяет создавать более крупные и выразительные языковые модели. Каждая "голова внимания" может использоваться для изучения различных отношений между словами, что улучшает выполнение задач обработки естественного языка.

**BERT** (Bidirectional Encoder Representations from Transformers) — это очень большая многослойная трансформерная сеть с 12 слоями для *BERT-base* и 24 для *BERT-large*. Модель сначала предварительно обучается на большом корпусе текстовых данных (WikiPedia + книги) с использованием обучения без учителя (предсказание замаскированных слов в предложении). Во время предварительного обучения модель усваивает значительный уровень понимания языка, который затем можно использовать с другими наборами данных с помощью тонкой настройки. Этот процесс называется **трансферным обучением**.

![Изображение с сайта http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362e39ee4381aab7cad06b5465a0b5f053a0f2aa05fbe14e746.ru.png)

Существует множество вариаций архитектур трансформеров, включая BERT, DistilBERT, BigBird, OpenGPT3 и другие, которые можно тонко настраивать. Пакет [HuggingFace](https://github.com/huggingface/) предоставляет репозиторий для обучения многих из этих архитектур с использованием PyTorch.

## Использование BERT для классификации текста

Давайте посмотрим, как мы можем использовать предварительно обученную модель BERT для решения нашей традиционной задачи: классификации последовательностей. Мы будем классифицировать наш оригинальный набор данных AG News.

Сначала загрузим библиотеку HuggingFace и наш набор данных:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Поскольку мы будем использовать предварительно обученную модель BERT, нам потребуется использовать определенный токенайзер. Сначала мы загрузим токенайзер, связанный с предварительно обученной моделью BERT.

Библиотека HuggingFace содержит репозиторий предварительно обученных моделей, которые можно использовать, просто указав их имена в качестве аргументов для функций `from_pretrained`. Все необходимые бинарные файлы для модели будут автоматически загружены.

Однако иногда может потребоваться загрузить собственные модели. В этом случае вы можете указать директорию, содержащую все соответствующие файлы, включая параметры для токенайзера, файл `config.json` с параметрами модели, бинарные веса и т. д.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Объект `tokenizer` содержит функцию `encode`, которую можно напрямую использовать для кодирования текста:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Затем давайте создадим итераторы, которые мы будем использовать во время обучения для доступа к данным. Поскольку BERT использует свою собственную функцию кодирования, нам нужно будет определить функцию дополнения, аналогичную `padify`, которую мы определили ранее:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

В нашем случае мы будем использовать предварительно обученную модель BERT под названием `bert-base-uncased`. Давайте загрузим модель с помощью пакета `BertForSequenceClassfication`. Это гарантирует, что наша модель уже имеет необходимую архитектуру для классификации, включая финальный классификатор. Вы увидите предупреждающее сообщение, в котором говорится, что веса финального классификатора не инициализированы, и модель потребует предварительного обучения - это совершенно нормально, потому что именно этим мы собираемся заняться!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Теперь мы готовы начать обучение! Поскольку BERT уже предварительно обучен, мы хотим использовать довольно маленькую скорость обучения, чтобы не испортить начальные веса.

Всю основную работу выполняет модель `BertForSequenceClassification`. Когда мы вызываем модель на обучающих данных, она возвращает как значение потерь (loss), так и выходные данные сети для входного минибатча. Мы используем значение потерь для оптимизации параметров (`loss.backward()` выполняет обратное распространение ошибки), а `out` — для вычисления точности обучения, сравнивая полученные метки `labs` (вычисленные с помощью `argmax`) с ожидаемыми `labels`.

Чтобы контролировать процесс, мы накапливаем значения потерь и точности за несколько итераций и выводим их каждые `report_freq` циклов обучения.

Это обучение, скорее всего, займет довольно много времени, поэтому мы ограничиваем количество итераций.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Вы можете заметить (особенно если увеличить количество итераций и подождать достаточно долго), что классификация с использованием BERT дает довольно хорошую точность! Это связано с тем, что BERT уже достаточно хорошо понимает структуру языка, и нам нужно лишь дообучить финальный классификатор. Однако, поскольку BERT — это большая модель, весь процесс обучения занимает много времени и требует значительных вычислительных ресурсов! (GPU, и желательно больше одного).

> **Note:** В нашем примере мы использовали одну из самых маленьких предобученных моделей BERT. Существуют более крупные модели, которые, вероятно, дадут лучшие результаты.


## Оценка производительности модели

Теперь мы можем оценить производительность нашей модели на тестовом наборе данных. Цикл оценки очень похож на цикл обучения, но не забудьте переключить модель в режим оценки, вызвав `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Основные выводы

В этом разделе мы увидели, как легко взять предварительно обученную языковую модель из библиотеки **transformers** и адаптировать её для задачи классификации текста. Аналогично, модели BERT могут использоваться для извлечения сущностей, ответа на вопросы и других задач обработки естественного языка.

Модели трансформеров представляют собой передовой уровень в NLP, и в большинстве случаев они должны быть первым решением, с которого вы начинаете эксперименты при реализации пользовательских NLP-решений. Однако понимание базовых принципов рекуррентных нейронных сетей, обсуждаемых в этом модуле, крайне важно, если вы хотите создавать продвинутые нейронные модели.



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
